# 1. Neural Network Training

In [1]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler

from Ranker.neuralnetwork import NeuralNetwork
from DataHandling.train_data import load_data


### 1.1. Load training and test data

In [2]:
feature_indices = [108, 18, 103, 3, 8, 13, 125, 126, 107, 17, 102, 2, 7, 12, 105, 15, 100, 0, 5, 10]

In [3]:
# load training data
X_train, y_train = load_data(path="../../../data/MSLR-WEB10K/Fold1/train.txt", nrows=100000, feature_indices=feature_indices)
#X_test, y_test = load_data(path="../../../data/MSLR-WEB10K/Fold1/test.txt", nrows=10000)



In [4]:
y_trans = torch.zeros(len(y_train), 5)
for i, label in enumerate(y_train):
    y_trans[i, 0:int(label)+1] = 1 


### 1.2. Create model

In [5]:
model = NeuralNetwork(n_features=len(feature_indices), n_hidden=10)

In [6]:
### define optimizer and loss fct
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss = nn.CrossEntropyLoss()

### 1.3. Train Model

In [7]:
for epoch in range(10):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = model.train_loop(torch.Tensor(X_train), y_trans, loss, optimizer)

Epoch 1
-------------------------------
loss: 3.982523  [   32/100000]
loss: 1.508958  [16032/100000]
loss: 1.252131  [32032/100000]
loss: 1.984476  [48032/100000]
loss: 1.227871  [64032/100000]
loss: 1.272001  [80032/100000]
loss: 1.975010  [96032/100000]
Epoch 2
-------------------------------
loss: 1.291760  [   32/100000]
loss: 1.621499  [16032/100000]
loss: 1.646893  [32032/100000]
loss: 1.719767  [48032/100000]
loss: 1.152898  [64032/100000]
loss: 1.149831  [80032/100000]
loss: 1.846484  [96032/100000]
Epoch 3
-------------------------------
loss: 1.356960  [   32/100000]
loss: 1.694168  [16032/100000]
loss: 1.156440  [32032/100000]
loss: 1.314789  [48032/100000]
loss: 1.735611  [64032/100000]
loss: 1.710166  [80032/100000]
loss: 0.989067  [96032/100000]
Epoch 4
-------------------------------
loss: 1.339719  [   32/100000]
loss: 1.472140  [16032/100000]
loss: 2.378346  [32032/100000]
loss: 1.818106  [48032/100000]
loss: 1.080953  [64032/100000]
loss: 1.975844  [80032/100000]
los

### 1.4. Save Model

In [8]:
# save mdoel
model.save("../models/nn.pth")

model saved in  ../models/nn_2.pth
